#####  데이터 불러오기 실습

In [180]:
from torchvision import datasets, models, transforms

In [181]:
data_transforms=transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                          [0.5,0.5,0.5])
])

In [182]:
import os
path='C:/python/Cats_and_dogs/training_set/training_set'
path1='C:/python/Cats_and_dogs/test_set/test_set'


In [183]:
training_datasets=datasets.ImageFolder(path,data_transforms)
test_datasets=datasets.ImageFolder(path1,data_transforms)


In [184]:

class_names=training_datasets.classes
print(class_names)

['cats', 'dogs']


In [185]:
train_loader=torch.utils.data.DataLoader(training_datasets,
batch_size=64,shuffle=True)

test_loader=torch.utils.data.DataLoader(test_datasets,
batch_size=64,shuffle=True)


##### device를 설정해서 GPU 연산을 가능하게 해주는 CUDA를 
##### 사용하자. cpu라고 뜨면 코랩이면 런타임에서 gpu로
#### 설정을 바꿔준다.

In [186]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [187]:
learning_rate = 0.001
training_epochs = 2
batch_size = 100

In [188]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        # L1 ImgIn shape=(?, 224, 224, 1)
        #    Conv     -> (?, 224, 224, 32)
        #    Pool     -> (?, 112, 112, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 112, 112, 32)
        #    Conv      ->(?, 112, 112, 64)
        #    Pool      ->(?, 56, 56, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 56, 56, 64)
        #    Conv      ->(?, 56, 56, 128)
        #    Pool      ->(?, 28, 28, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(28 * 28 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 2, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [189]:
# CNN 모델 정의
model = CNN().to(device)

In [190]:
## 비용함수와 옵티마이저를 정의합니다.
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [191]:
## 총 배치의 수를 출력해보겠습니다.

total_batch=len(train_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 126


In [192]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))








[Epoch:    1] cost = 0.844974756
[Epoch:    2] cost = 0.693263888


##### 이제 테스트 해보겠습니다.

In [194]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()

correct=0
total=0

with torch.no_grad():

    for data in test_loader:
        images, labels=data[0].to(device), data[1].to(device)
        outputs=model(images)
        _, predicted=torch.max(outputs.data, 1)
        total +=labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'accuracy of 10000 test images: {100*correct/total}%')




accuracy of 10000 test images: 49.67869500741473%
